In [ ]:
import torch
import numpy as np
import torchvision
import torch.nn as nn
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((227, 227)),  
    torchvision.transforms.ToTensor(),       
    torchvision.transforms.Normalize(mean=(0.5,), std=(0.5,)) 
])

train_dataset = torchvision.datasets.CIFAR10(
    root='/home/kami/Documents/datasets/',
    train=True,
    download=False,
    transform=transform
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True
)


for images, labels in train_loader:
    print(f"Batch shape: {images.shape}")  
    print(f"Labels shape: {labels.shape}") 
    print(f"Image tensor min: {images.min()}, max: {images.max()}")  
    break  